<h1>This notebook is specialize for the IBM Capstone project<h1>

In [1]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


<h2>Segmenting and Clustering Neighborhoods in Toronto<h2>

**Week 3 Peer Graded Assignment**<br>
**In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto based on the postalcode and borough information.. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.**

**For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas  dataframe so that it is in a structured format like the New York dataset.**

**Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.**

*Here I downloaded Toronto demographics data from kaggle, which contains geographics data, so I dont need to grab it from Wiki* <br>
[Toronto data](https://www.kaggle.com/youssef19/toronto-neighborhoods-inforamtion)

In [54]:
import requests
import folium
from sklearn.cluster import KMeans
tt=pd.read_csv('D:/anaconda/udemy/Toronto_neighborhood_demographics_geographics_venues.csv')
tt.head()
tt=tt[['Neighborhood','long_latt']]
nei=tt['Neighborhood']
long=[]
latt=[]
for r in tt.long_latt:
    r=r.split(',')
    long.append(float(r[0][1:]))
    latt.append(float(r[-1][:-1]))
df=pd.DataFrame(columns=['Neighborhoods','lattitude','longitude'])
df['Neighborhoods']=nei
df['longitude']=long
df['lattitude']=latt
toronto_loc=[43.651070,-79.347015]
t=folium.Map(location=toronto_loc)
for i in zip(df['Neighborhoods'],latt,long):
    folium.Popup(i[0])
    folium.CircleMarker(
    [i[1],i[2]],
    radius=5,
    popup=i[0],
    fill=True,
    color='green').add_to(t)


In [161]:
CLIENT_ID = '5YJAC3OAHPTQC05P2KPFJA0E4S504O5W3CPMZWUBKKCSH2SE' # your Foursquare ID
CLIENT_SECRET = 'YYLDTOPWE5ARNTKZU4OZA4JABAZEEFIKRFKXS5KJZFMCSULE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius=500

# df_n=pd.DataFrame(columns=['Neighborhood','Neighborhood_latt','Neighborhood_long','venue','venue_category','venue_latt',
#                           'venue_long'])
venue_list=[]
for i in range(df.shape[0]):
    nei_name=df.loc[i,'Neighborhoods']
    n_latt=df.loc[i,'lattitude']
    n_long=df.loc[i,'longitude']
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        n_latt, 
        n_long, 
        radius, 
        LIMIT)
    result=requests.get(url).json()
    for j in range(len(result['response']['groups'][0]['items'])):
        venue_name=result['response']['groups'][0]['items'][j]['venue']['name']
        venue_cate=result['response']['groups'][0]['items'][j]['venue']['categories'][0]['name']
        venue_latt=result['response']['groups'][0]['items'][j]['venue']['location']['lat']
        venue_long=result['response']['groups'][0]['items'][j]['venue']['location']['lng']
        row=[nei_name,n_latt,n_long,venue_name,venue_cate,venue_latt,venue_long]
        venue_list.append(row)
df_n=pd.DataFrame(venue_list,columns=['Neighborhood','Neighborhood_latt','Neighborhood_long','venue','venue_category','venue_latt',
                          'venue_long'])
df_n
#test_code
# nei_name=df.loc[0,'Neighborhoods']
# n_latt=df.loc[0,'lattitude']
# n_long=df.loc[0,'longitude']
# url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#         CLIENT_ID, 
#         CLIENT_SECRET, 
#         VERSION, 
#         n_latt, 
#         n_long, 
#         radius, 
#         LIMIT)
# result=requests.get(url).json()
# result['response']['groups'][0]['items']



,Neighborhood,Neighborhood_latt,Neighborhood_long,venue,venue_category,venue_latt,venue_long
0,Agincourt North,43.797406,-79.281616,Shell,Gas Station,43.797544,-79.283345
1,Agincourt North,43.797406,-79.281616,Subway,Sandwich Place,43.797503,-79.282181
2,Agincourt North,43.797406,-79.281616,Havendale Park,Park,43.796010,-79.284398
3,Agincourt North,43.797406,-79.281616,凱聲,Asian Restaurant,43.801003,-79.283363
4,Agincourt South-Malvern West,43.785187,-79.289169,Congee Queen 皇后名粥,Cantonese Restaurant,43.783910,-79.292319
...,...,...,...,...,...,...,...
2228,Yorkdale-Glen Park,43.725320,-79.470658,Parker Pet Care,Pet Store,43.723333,-79.474341
2229,Yorkdale-Glen Park,43.725320,-79.470658,Lac Vien Vietnamese Restaurant,Vietnamese Restaurant,43.721259,-79.468472
2230,Yorkdale-Glen Park,43.725320,-79.470658,Kaboom Fireworks,Fireworks Store,43.725409,-79.466277
2231,Yorkdale-Glen Park,43.725320,-79.470658,toronto sport and social club,Gym / Fitness Center,43.725111,-79.465766


In [171]:
df_dummy=pd.get_dummies(df_n['venue_category'])
df_dummy.drop(columns='Neighborhood',inplace=True)
df_dummy['Neighborhood']=df_n['Neighborhood']
df_dummy['Neighborhood_latt']=df_n['Neighborhood_latt']
df_dummy['Neighborhood_long']=df_n['Neighborhood_long']
df_dummy=df_dummy[list(df_dummy.columns[-3:])+list(df_dummy.columns[0:-3])]
df_dummy

,Neighborhood,Neighborhood_latt,Neighborhood_long,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Arcade,Arepa Restaurant,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt North,43.797406,-79.281616,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Agincourt North,43.797406,-79.281616,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Agincourt North,43.797406,-79.281616,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Agincourt North,43.797406,-79.281616,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Agincourt South-Malvern West,43.785187,-79.289169,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2228,Yorkdale-Glen Park,43.725320,-79.470658,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2229,Yorkdale-Glen Park,43.725320,-79.470658,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2230,Yorkdale-Glen Park,43.725320,-79.470658,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2231,Yorkdale-Glen Park,43.725320,-79.470658,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [236]:
df_gp=df_dummy.groupby(by='Neighborhood').mean()


df_gp
top_list=[]
df_gp.loc['Agincourt North',df_gp.columns[2:]].sort_values(ascending=False)[0:5].index
for i in df_gp.index:
    v_list=df_gp.loc[i,df_gp.columns[2:]].sort_values(ascending=False)[0:5].index.to_list()
    v_list.insert(0,df_gp.loc[i,df_gp.columns[1]])
    v_list.insert(0,df_gp.loc[i,df_gp.columns[0]])
    v_list.insert(0,i)
    top_list.append(v_list)
df_venue=pd.DataFrame(top_list,columns=['Neighborhood','Neighborhood_latt','Neighborhood_long','1th','2th','3th','4th','5th'])
df_venue
df_gp

,Neighborhood_latt,Neighborhood_long,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Arcade,Arepa Restaurant,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Agincourt North,43.797406,-79.281616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Agincourt South-Malvern West,43.785187,-79.289169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Alderwood,43.595500,-79.553204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Annex,43.674431,-79.412147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.030303,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Banbury-Don Mills,43.732570,-79.326505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wychwood,43.673910,-79.414654,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.037037,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Yonge-Eglinton,43.703398,-79.397644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.020408,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020408
Yonge-St.Clair,43.695979,-79.405012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [248]:
kmean=KMeans(n_clusters=5,random_state=0).fit(df_gp)
kmean
labels=pd.DataFrame(kmean.labels_,index=df_gp.index)
labels.rename(columns={0:'cluster'},inplace=True)
df_gp=df_gp[[df_gp.columns[-1]]+list(df_gp.columns[0:-1])]
df_gp


,cluster,Neighborhood_latt,Neighborhood_long,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Arcade,Arepa Restaurant,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Agincourt North,4,43.797406,-79.281616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Agincourt South-Malvern West,0,43.785187,-79.289169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.043478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Alderwood,4,43.595500,-79.553204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Annex,0,43.674431,-79.412147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.030303,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Banbury-Don Mills,0,43.732570,-79.326505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wychwood,0,43.673910,-79.414654,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.037037,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
Yonge-Eglinton,0,43.703398,-79.397644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.020408,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020408
Yonge-St.Clair,4,43.695979,-79.405012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [253]:
t2=folium.Map(location=toronto_loc)
color=['red','green','blue','purple','beige']
for i in df_gp.index:
    folium.Marker(
    location=[df_gp.loc[i,'Neighborhood_latt'],df_gp.loc[i,'Neighborhood_long']],
    popup=i+'\n'+'cluster'+str(df_gp.loc[i,'cluster']),
    icon=folium.Icon(color=color[df_gp.loc[i,'cluster']])).add_to(t2)
t2